In [2]:
%run old_code/code.py
latex.add_to_preamble(r"\usetikzlibrary{backgrounds}")
Partitions.options.notation = "french"

Compiling ./old_code/code1.pyx...


In [3]:
@cached_function
def PP(n):
    S = SymmetricFunctions(ZZ)
    s = S.s()
    m = S.m()
    P = ZZ['X,Y,Z']
    XYZ = P.gens()
    Etuple = sage.structure.parent.Set_PythonType(sage.rings.polynomial.polydict.ETuple)
    PP = CombinatorialFreeModule(ZZ, Etuple, key=n)
    def to_poly(p):
        return PP.sum_of_terms( [m.exponents()[0], c] for c,m in list(p.expand(3, XYZ)))
    phi = SetMorphism(Hom(s, PP), to_poly)
    phi.register_as_coercion()
    return PP

In [35]:
from sage.categories.morphism import SetMorphism
def dimensions(n, r, left="s", right="s"):
    H = harmonic_bicharacter(n)
    S = SymmetricFunctions(ZZ)
    s = S.s()
    m = S.m()
    
    if left == "s":
        left = s
    elif left == "P":
        left = PP(n)
    else:
        left = m

    H = tensor([left,s])(H)
    
    dimensions = {}
    for (l, mu), c in H:
        lp = tuple(l)+(0,)*(r-len(l))
        if right == "dim":
            value = c * StandardTableaux(mu).cardinality()
        else:
            value = s.term(mu, c)
        dimensions[lp] = dimensions.get(lp, 0) + value
    return dimensions

In [36]:
dimensions(3,3, left="P", right="dim")

{(0, 0, 0): 1,
 (0, 0, 1): 2,
 (0, 0, 2): 2,
 (0, 0, 3): 1,
 (0, 1, 0): 2,
 (0, 1, 1): 3,
 (0, 1, 2): 1,
 (0, 2, 0): 2,
 (0, 2, 1): 1,
 (0, 3, 0): 1,
 (1, 0, 0): 2,
 (1, 0, 1): 3,
 (1, 0, 2): 1,
 (1, 1, 0): 3,
 (1, 1, 1): 1,
 (1, 2, 0): 1,
 (2, 0, 0): 2,
 (2, 0, 1): 1,
 (2, 1, 0): 1,
 (3, 0, 0): 1}

In [67]:
vars = SR.var('x,y,z')

color = {0: "red", 1: "blue"}.__getitem__
def latex_sym(value):
    if value in ZZ:
        return latex(value)
    return "+".join(r"\raisebox{\depth}{\scalebox{.3}{%s}}"%latex(p) for p,c in value)
def node_label(lp, value):
    return latex(prod(x^d for x,d in zip(vars,lp)))+": "+latex_sym(value)
def picture(dimensions):
    def key(lp):
        return "".join(str(i) for i in lp)
    s = r"\begin{tikzpicture}"+"\n"+r"  \tikzstyle{every node}=[fill=white]"+"\n"
    for lp, value in dimensions.items():
        y = 2*sum(lp) + lp[1]/3
        x = 2*(lp[2] - lp[0])
        s = s + r"  \node (%s) at (%s,%s) {$%s$};"%(key(lp), x,y,node_label(lp, value))+"\n"
    s = s +r"  \begin{pgfonlayer}{background}"+"\n"
    for lp in dimensions.keys():
        for i in range(2):
            for l0 in range(1,lp[i]+1):
                lp2 = list(lp);
                lp2[i] = lp2[i] - l0
                lp2[i+1] += 1
                lp2 = tuple(lp2)
                style =  "thin" if (l0 > 1) else "thick"
                if lp2 not in dimensions:
                    continue
                s = s + r"  \draw[->,%s,%s] (%s) -- (%s);"%(color(i), style, key(lp), key(lp2)) +"\n"
        if lp[0] > 0 and all(l==0 for l in lp[1:]):
            lp2 = (lp[0]-1,) + lp[1:]
            style = "thin"
            s = s + r"  \draw[->,%s,%s] (%s) -- (%s);"%("black", style, key(lp), key(lp2)) +"\n"
    s = s + r"  \end{pgfonlayer}"+"\n"+r"\end{tikzpicture}"
    return s

In [68]:
view(LatexExpr(picture(dimensions(3,3, left="P", right="dim"))))

In [69]:
import io, os
def pdf(file, tikz):
    io.open(file+".tikz", "w").write(unicode(tikz))
    os.system("tikz2pdf %s.tikz"%file)
    os.system("pdf2svg %s.pdf %s.svg"%(file, file))

## Production of pictures

In [70]:
pdf("harmonic_graph-3-3-P", picture(dimensions(3,3, left="P", right="dim")))

In [71]:
pdf("harmonic_graph-3-3-m", picture(dimensions(3,3, left="m", right="dim")))

In [72]:
pdf("harmonic_graph-3-3-s", picture(dimensions(3,3, left="s", right="dim")))

In [74]:
pdf("harmonic_graph-3-3-P-s", picture(dimensions(3,3, left="P")))